本次作业以垃圾邮件分类任务为基础，要求提取文本特征并使用朴素贝叶斯算法进行垃圾邮件识别（调用已有工具包或自行实现）。

### 任务介绍
电子邮件是互联网的一项重要服务，在大家的学习、工作和生活中会广泛使用。但是大家的邮箱常常被各种各样的垃圾邮件填充了。有统计显示，每天互联网上产生的垃圾邮件有几百亿近千亿的量级。因此，对电子邮件服务提供商来说，垃圾邮件过滤是一项重要功能。而朴素贝叶斯算法在垃圾邮件识别任务上一直表现非常好，至今仍然有很多系统在使用朴素贝叶斯算法作为基本的垃圾邮件识别算法。

本次实验数据集来自[Trec06](https://plg.uwaterloo.ca/cgi-bin/cgiwrap/gvcormac/foo06)的中文垃圾邮件数据集，目录解压后包含三个文件夹，其中data目录下是所有的邮件（未分词），已分词好的邮件在data_cut目录下。邮件分为邮件头部分和正文部分，两部分之间一般有空行隔开。标签数据在label文件夹下，文件中每行是标签和对应的邮件路径。‘spam’表示垃圾邮件，‘ham’表示正常邮件。

本次实验

基本要求：
1. 提取正文部分的文本特征；
2. 划分训练集和测试集（可以借助工具包。一般笔记本就足够运行所有数据，认为实现困难或算力不够的同学可以采样一部分数据进行实验。）；
3. 使用朴素贝叶斯算法完成垃圾邮件的分类与预测，要求测试集准确率Accuracy、精准率Precision、召回率Recall均高于0.9（本次实验可以使用已有的一些工具包完成如sklearn）；
4. 对比特征数目（词表大小）对模型效果的影响；
5. 提交代码和实验报告。

扩展要求：
1. 邮件头信息有时也可以协助判断垃圾邮件，欢迎学有余力的同学们尝试；
2. 尝试自行实现朴素贝叶斯算法细节；
3. 尝试对比不同的概率计算方法。

### 导入工具包

In [1]:
import random # 随机相关包
import numpy as np # 常用数学运算工具包
from tqdm.autonotebook import tqdm # 进度条工具包
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # 提取文本特征向量的类
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, ComplementNB # 三种朴素贝叶斯算法，差别在于估计p(x|y)的方式
from sklearn.model_selection import train_test_split, cross_validate # 划分数据集函数
from sklearn.metrics import accuracy_score, precision_score, recall_score # 评价指标
from scipy.sparse import hstack
import re
RANDOM_SEED = 2020 # 固定随机种子

<ipython-input-1-51c0f271ad32>:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm # 进度条工具包


### 读入数据

In [2]:
data_dir = './trec06c-utf8/' # 数据集目录
label_file = './trec06c-utf8/label/index' # 标签文件路径

In [3]:
def read_file(path): # 读取一个邮件文件，返回邮件头和正文信息
    with open(path, 'r') as in_f: # 读入文件
        lines = [line.strip() for line in in_f] # 去掉换行符
        head, text = [], []
        l = head # 刚开始都是邮件头
        for line in lines: # 遍历每一行
            if line == '': # 如果是空行
                l = text # 则表示是正文部分
                continue # 跳过空行
            l.append(line) # 以list返回，list中每个元素是一行文本
    return head, text

labels, heads, texts = [], [], [] 
with open(label_file, 'r') as in_f: # 读入标签文件
    lines = [line.strip() for line in in_f if line.strip() != ''] # 读入所有非空行，并去掉换行符
#     lines = random.sample(lines, 10000) # 如果处理不了所有数据量可采样一部分
    for line in tqdm(lines, leave=False, mininterval=1, desc='read file'): # 遍历每一行
        label, path = line.split() # 分割为标签和文件路径
        label = 1 if label == 'spam' else 0 # 标签转化成01，垃圾邮件为1
        path = data_dir + path.replace('../data', 'data_cut') # 路径转化为正确的相对路径
        head, text = read_file(path) # 读入头信息和正文文本
        labels.append(label)
        heads.append(head) # 头信息继续按list形式存储
        texts.append(' '.join(text)) # 正文文本信息将每行连接起来，空格隔开

read file:   0%|          | 0/64620 [00:00<?, ?it/s]

### 提取头信息中的特征

In [4]:
f_repeat = 1 # 为方便处理，直接将提取的特征作为文本词拼接在正文最后，并重复多次以增强特征权重。也可暂时不加权，变成稀疏向量后再加权
x_priorities = [] # 提取了邮件优先级。
for head in heads: # 遍历每个邮件对头信息
    x_priority = '' # 初始化优先级为空
    for line in head: # 对头信息中每一行
        if 'x-priority' in line.lower(): # 如果是优先级信息，则去除空格记录
            x_priority = 'priority' + re.search('x-priority: ([a-z0-9]+)', line.lower()).group(1)
    x_priorities.append(' '.join([x_priority]*f_repeat) if x_priority != '' else '')

# 拼接到每个邮件正文的最后
extend_texts = [' '.join([texts[i], x_priorities[i]]) for i in range(len(texts))]

In [5]:
# 也可以先分开存储，变成稀疏向量后再拼接
header_texts = [(texts[i], x_priorities[i]) for i in range(len(texts))]

### 划分训练集和测试集

In [6]:
# 划分训练集和测试集，随机取80%作为训练集，20%作为测试集。
# text_train, text_test, label_train, label_test = train_test_split(texts, labels, test_size=0.2, random_state=RANDOM_SEED)
text_train, text_test, label_train, label_test = train_test_split(extend_texts, labels, test_size=0.2, random_state=RANDOM_SEED)

# 使用sklearn的文本向量化工具，max_df=0.9表示过滤一些在90%的文档中都出现过的词，min_df=10表示在所有文档中出现低于10次的词。
# 该类可以自动按空格分词，并将list中每个字符串转化为一个稀疏向量，向量共有M个元素表示M个词出现的次数，M为特征字典的大小。
vectorizer = CountVectorizer(max_df=0.9, min_df=10)
# vectorizer = TfidfVectorizer(max_df=0.9, min_df=10) # 文本向量每个元素是该词的tf-idf而不是次数。
x_train = vectorizer.fit_transform(text_train) # 传入所有训练文本会返回一个稀疏矩阵，总共N行表示N个样本，M行表示M个特征
x_test = vectorizer.transform(text_test)
y_train, y_test = np.array(label_train), np.array(label_test) # 标签
print(x_train.shape, x_test.shape) # 输出矩阵大小
print('priority1' in vectorizer.vocabulary_)
print('priority2' in vectorizer.vocabulary_)

(51696, 41455) (12924, 41455)
True
True


In [7]:
# 也可以变成稀疏向量后再拼接
text_train, text_test, label_train, label_test = train_test_split(header_texts, labels, test_size=0.2, random_state=RANDOM_SEED)

vectorizer_text = CountVectorizer(max_df=0.9, min_df=10) # 转化正文部分
x_train_text = vectorizer_text.fit_transform([t[0] for t in text_train])
x_test_text = vectorizer_text.transform([t[0] for t in text_test])
print(x_train_text.shape)

vectorizer_h1 = CountVectorizer(min_df=0.0, max_df=1.0) # 转化头特征1
x_train_h1 = vectorizer_h1.fit_transform([t[1] for t in text_train])
x_test_h1 = vectorizer_h1.transform([t[1] for t in text_test])
print(x_train_h1.shape)
print(vectorizer_h1.vocabulary_)
print(vectorizer_h1.stop_words_)

x_train = hstack([x_train_text, x_train_h1 * f_repeat]) # 拼接稀疏向量，也可在此对特征加权
x_test = hstack([x_test_text, x_test_h1 * f_repeat]) # 拼接稀疏向量，也可在此对特征加权
print(x_train.shape)


(51696, 41451)
(51696, 6)
{'priority3': 3, 'priority2': 2, 'priority4': 4, 'priority1': 1, 'priority5': 5, 'priority0': 0}
set()
(51696, 41457)


### 训练和测试模型

In [8]:
for NB in [BernoulliNB(), MultinomialNB(), ComplementNB()]: # 测试三种类型的朴素贝叶斯
    NB.fit(x_train, y_train) # 在训练集上训练
    p_test = NB.predict(x_test) # 在测试集上预测，获得预测值
    test_acc = accuracy_score(y_test, p_test) # 将测试预测值与测试集标签对比获得准确率
    test_precision = precision_score(y_test, p_test) # 精准率，判断为1的邮件中有多少真的为垃圾邮件，垃圾邮件分类任务中的重要指标，因为不希望将非垃圾邮件判为垃圾邮件
    test_recall = recall_score(y_test, p_test) # 召回率，真的垃圾邮件中有多少被识别出来
    print(NB)
    print('accuracy: {:.4f}, precision: {:.4f}, recall: {:.4f}'.format(test_acc, test_precision, test_recall)) # 输出评价指标

BernoulliNB()
accuracy: 0.9208, precision: 0.9819, recall: 0.8967
MultinomialNB()
accuracy: 0.9738, precision: 0.9798, recall: 0.9807
ComplementNB()
accuracy: 0.9732, precision: 0.9799, recall: 0.9795


### 小结
在本数据集及特征环境下，朴素贝叶斯文本分类的多项式模型MultinomialNB和补足模型ComplementNB表现较好，他们考虑了词出现的频率，伯努利模型只考虑是否出现。提取的两个头信息特征对效果有略微提升。文本向量化的两种方式次数向量CountVectorizer和tf-idf向量TfidfVectorizer与不同模型结合有略微差别。

作业问题：
1. 6万多邮件只是很小规模的数据，写代码要考虑效率，不完全是机器的问题；
2. 词筛选：停用词、词频、tf-idf、信息增益、互信息等
3. 特征概率连续相乘会出现精度不足的问题，可以对概率取对数后相加: $argmax_y \{logP(y)+\sum_i logP(x_i|y)\}$；
4. 比较贝叶斯公式分子即可，分母$P(x_1,x_2...)$每个类别都一样可忽略，但计算出的概率和就不为1了；
5. 不加先验概率P(y)其实有点类似极大似然估计
6. 错误概率估计举例（特征和不为1，可能会受邮件长度影响）：  
    $P(x_i|y) = y类别中x_i出现总次数/y中邮件总数$  
    $P(x_i|y) = y类别中包含x_i的邮件数/y类别中总词数$
7. 平滑项（假设存在两封虚拟邮件权重为$\alpha$，一封邮件每个词各一个，和一封空邮件）：  
    多项式$P(x_i|y) = (y类别中x_i出现总次数 + \alpha)/ (y类别中所有特征出现总次数+\alpha*总特征数)$使得$\sum_i P(x_i|y)=1$  
    伯努利$P(x_i|y) = (y类别中包含x_i的邮件数 + \alpha)/ (y类别中邮件数+\alpha*2)$使得$P(x_i=0|y)+P(x_i=1|y)=1$ 